# Neural network Spam filtering using forward backward propagation

Used Ling spam data set from the links provided in the assignment statements

# Data extraction

In [258]:
#Required libraires for the project
import os
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import re
train_text_list=[]
train_label_list=[]
test_text_list=[]
test_label_list=[]
#created train and test datasets from spam datset
train_df = pd.DataFrame(columns=['Text','Label'])
test_df = pd.DataFrame(columns=['Text','Label'])
count=0
spam_count=0
non_spam_count=0
for root, dirs, files in os.walk("Train/"):
        for  root, dirs, files in os.walk(root):
            for file in files:
                #if (count > 0):
                    f = open(root+"/"+file, "r",encoding='latin-1', errors='ignore')
                    text= f.read()
                    if (file[0:4]=="spms"):
                        lable= "spam"
                        spam_count=spam_count+1
                    else:
                        lable= "not_spam"
                        non_spam_count=non_spam_count+1
                    f.close()
                    train_text_list.append(text[8:])
                    train_label_list.append(lable)

train_df['Text']=train_text_list
train_df['Label']=train_label_list
#Creating test into different data frame
for root, dirs, files in os.walk("Test/"):
        for  root, dirs, files in os.walk(root):
            for file in files:
                #if (count > 0):
                    f = open(root+"/"+file, "r",encoding='latin-1', errors='ignore')
                    text= f.read()
                    if (file[0:4]=="spms"):
                        lable= "spam"
                        spam_count=spam_count+1
                    else:
                        lable= "not_spam"
                        non_spam_count=non_spam_count+1
                    f.close()
                    test_text_list.append(text[8:])
                    test_label_list.append(lable)
test_df['Text']=test_text_list
test_df['Label']=test_label_list

# Start of preprocessing

In [259]:
#Made text to lower case in train and test as apart of prep processing
train_df['Text'] = [entry.lower() for entry in train_df['Text']]
test_df['Text'] = [entry.lower() for entry in test_df['Text']]

In [260]:
#display tail (end 5 entries of train data)
train_df.tail()

,Text,Label
944,call : field report / endanger lg . lsa ' 96\...,not_spam
945,re : 6 . 139 word own opposite\n\ncurious cas...,not_spam
946,summer school program\n\ninternational summer...,not_spam
947,web development offer . . . open immediately\...,spam
948,summary sociolinguistic course syllabus\n\nde...,not_spam


In [261]:
#display tail (end 5 entries of test data)
test_df.tail()

,Text,Label
294,sla conference pari 1999\n\ncall for papers x...,not_spam
295,conference announcement\n\nsouthern illinoi u...,not_spam
296,semantic\n\nwe would like to bring to your at...,not_spam
297,released vol . 2\n\nreleased ! ! ! introducin...,spam
298,web development offer . . . open immediately\...,spam


In [262]:
#Added one more column to get number if of each row in data frame
ind_list=[]
ind_count=0
for i in train_df['Text']:
    ind_count=ind_count+1
    ind_list.append(ind_count)
train_df['Ind']= ind_list

In [263]:
#Added one more column to get number if of each row in data frame
ind_list1=[]
ind_count1=0
for i in test_df['Text']:
    ind_count1=ind_count1+1
    ind_list1.append(ind_count1)
test_df['Ind']= ind_list1

Data preprocessing and statistics

In [264]:
train_df.head()

,Text,Label,Ind
0,here\n\nuniversity degree programs increase p...,spam,1
1,bisfai deadline extension !\n\nbisfai deadlin...,not_spam,2
2,"incredible opportunity\n\ndear friend , recei...",spam,3
3,""" life without debt ""\n\npardon intrusion . o...",spam,4
4,"query :\n\ndear linguist , work double negati...",not_spam,5


In [265]:
print(train_df.shape)
print(train_df.Label.shape)

(949, 3)
(949,)


In [266]:
print(test_df.shape)
print(test_df.Label.shape)

(299, 3)
(299,)


based on labes got the count of spam and non spam

In [267]:
#Getting counts spam and not spam for preprocssing
spam_count=0
non_spam_count=0
for i in train_df['Label']:
    if (i== str('not_spam')):
        non_spam_count=non_spam_count+1
    if (i== str('spam')):
        spam_count= spam_count+1
        
print('non_spam_count', non_spam_count)
print('spam_count',spam_count)

non_spam_count 468
spam_count 481


In [268]:
#for just a check on data if the data frame is properly prepared
train_df['Label'].unique()

array(['spam', 'not_spam'], dtype=object)

In [269]:
train_df.describe(exclude=[np.number])#checking some stats of data

,Text,Label
count,949,949
unique,934,2
top,re :\n\n= 20 virtual girlfriend virtual boyfr...,spam
freq,4,481


In [270]:
#Making one more lab with 0 as Not spam and 1 as spam
train_df["label_tag"]= train_df['Label'].map( {'spam': 1, 'not_spam': 0} ).astype(int)
test_df["label_tag"]= train_df['Label'].map( {'spam': 1, 'not_spam': 0} ).astype(int)

Imported required modules for some staistics and to calculate the common words in the data set

In [271]:
#Imported required counters
from collections import Counter
import numpy as np
spam_counts = Counter()
not_spam_counts = Counter()
total_counts = Counter()
spam_not_ratios = Counter()

Applied word tokenize to convert the data into numbered features

In [272]:
#Word tokenize the raw text for train and test data as a part of preprocessing
from nltk import word_tokenize
train_df['Text'].dropna(inplace=True)
train_df['tokenized_text'] =train_df['Text'].apply(word_tokenize)
test_df['Text'].dropna(inplace=True)
test_df['tokenized_text'] =test_df['Text'].apply(word_tokenize)

Training data after tokenization to remove stop words and some special characters

In [273]:
train_df.head()

,Text,Label,Ind,label_tag,tokenized_text
0,here\n\nuniversity degree programs increase p...,spam,1,1,"[here, university, degree, programs, increase,..."
1,bisfai deadline extension !\n\nbisfai deadlin...,not_spam,2,0,"[bisfai, deadline, extension, !, bisfai, deadl..."
2,"incredible opportunity\n\ndear friend , recei...",spam,3,1,"[incredible, opportunity, dear, friend, ,, rec..."
3,""" life without debt ""\n\npardon intrusion . o...",spam,4,1,"[``, life, without, debt, ``, pardon, intrusio..."
4,"query :\n\ndear linguist , work double negati...",not_spam,5,0,"[query, :, dear, linguist, ,, work, double, ne..."


In [274]:
test_df.head()

,Text,Label,Ind,label_tag,tokenized_text
0,here\n\nuniversity degree programs increase p...,spam,1,1,"[here, university, degree, programs, increase,..."
1,"endanger language - edinburgh , sept 98 - cal...",not_spam,2,0,"[endanger, language, -, edinburgh, ,, sept, 98..."
2,nels 29 - - call for paper\n\n* * * * * * * *...,not_spam,3,1,"[nels, 29, -, -, call, for, paper, *, *, *, *,..."
3,kornfilt : turkish\n\njaklin kornfilt ( 1997 ...,not_spam,4,1,"[kornfilt, :, turkish, jaklin, kornfilt, (, 19..."
4,9th intl congress of linguist\n\nistituto di ...,not_spam,5,0,"[9th, intl, congress, of, linguist, istituto, ..."


In [275]:
#Remove speacial characters
#Check stop words and special characters
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
#special=['!','@','#','$','%','^','&','*','(',')',':','<','>',',','.',';','{','}','|','_','-','+','=','`','~',"--","'",'"','[',']','\n','/','\\']
special=[',','.',';','_','-','=','*']
for item  in train_df['tokenized_text']:
    item_new=[]
    for word in item:
        word=str(word)
        if (word in stop_words) or (word in special):
            item.remove(word)
for item  in test_df['tokenized_text']:
    item_new=[]
    for word in item:
        word=str(word)
        if (word in stop_words) or (word in special):
            item.remove(word)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prashanthimallijula/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [276]:
#Remove speacial characters
#Check stop words and special characters
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
#special=['!','@','#','$','%','^','&','*','(',')',':','<','>',',','.',';','{','}','|','_','-','+','=','`','~',"--","'",'"','[',']','\n','/','\\']
special=[',','.',';','_','-','=','*']
for item  in train_df['tokenized_text']:
    item_new=[]
    for word in item:
        word=str(word)
        if (word in stop_words) or (word in special):
            item.remove(word)
for item  in test_df['tokenized_text']:
    item_new=[]
    for word in item:
        word=str(word)
        if (word in stop_words) or (word in special):
            item.remove(word)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prashanthimallijula/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [277]:
train_df.head()

,Text,Label,Ind,label_tag,tokenized_text
0,here\n\nuniversity degree programs increase p...,spam,1,1,"[university, degree, programs, increase, perso..."
1,bisfai deadline extension !\n\nbisfai deadlin...,not_spam,2,0,"[bisfai, deadline, extension, !, bisfai, deadl..."
2,"incredible opportunity\n\ndear friend , recei...",spam,3,1,"[incredible, opportunity, dear, friend, receiv..."
3,""" life without debt ""\n\npardon intrusion . o...",spam,4,1,"[``, life, without, debt, ``, pardon, intrusio..."
4,"query :\n\ndear linguist , work double negati...",not_spam,5,0,"[query, :, dear, linguist, work, double, negat..."


In [278]:
test_df.head()

,Text,Label,Ind,label_tag,tokenized_text
0,here\n\nuniversity degree programs increase p...,spam,1,1,"[university, degree, programs, increase, perso..."
1,"endanger language - edinburgh , sept 98 - cal...",not_spam,2,0,"[endanger, language, edinburgh, sept, 98, call..."
2,nels 29 - - call for paper\n\n* * * * * * * *...,not_spam,3,1,"[nels, 29, call, paper, n, e, l, 29, annual, m..."
3,kornfilt : turkish\n\njaklin kornfilt ( 1997 ...,not_spam,4,1,"[kornfilt, :, turkish, jaklin, kornfilt, (, 19..."
4,9th intl congress of linguist\n\nistituto di ...,not_spam,5,0,"[9th, intl, congress, linguist, istituto, di, ..."


Calculate the required counts of spam and non spam and toal counts

# Feature engineering

For text data procesas Machine sing we need to convert the data into numeric for machine learning modelas run with numeric attributes 

In [279]:
#Getting spam and not spam word count from train data to train the model as a part feature enginerring
for i in range(len(train_df.Label)):
        if(train_df.at[i,'Label'] == 'spam'):
            for word in train_df.Text[i].split(" "):
                spam_counts[word] += 1
                total_counts[word] += 1
        else:
            for word in train_df.Text[i].split(" "):
                not_spam_counts[word] += 1
                total_counts[word] += 1

Below are the most common words in spam messages

In [280]:
print(spam_counts.most_common()[0:40])

[('_', 39231), ('.', 21396), ('*', 19064), ('-', 18913), (',', 12194), ('=', 9069), ('!', 6644), ('/', 4640), ('$', 4448), ("'", 3075), (':', 2834), ('"', 2667), ('+', 2660), ('0', 1958), ('20', 1770), (')', 1685), ('(', 1595), ('order', 1500), ('#', 1470), ('~', 1365), ('report', 1314), ('address', 1299), ('our', 1181), ('mail', 1171), ('email', 1152), ("'s", 1106), ('program', 1045), ('send', 1033), ('>', 992), ('free', 945), ('list', 938), ('receive', 886), ('money', 871), ('name', 868), ('?', 851), (';', 774), ('business', 751), ('3d', 736), ('one', 732), ('1', 731)]


Calculate spam and non spam ratio for eachspam words that are commonly used

In [281]:
for k,v in list(total_counts.most_common()):
    if(v > 100):
        spam_not_ratio = spam_counts[k] / float(not_spam_counts[k]+1)
        spam_not_ratios[k] = spam_not_ratio

Calculate the log value for ratio inorder to get the most used words as features

In [282]:
for word,percent in spam_not_ratios.most_common():
    if(percent > 1):
        spam_not_ratios[word] = np.log(percent)
    else:
        spam_not_ratios[word] = -np.log((1 / (percent+0.01)))

In [283]:
print(list(reversed(spam_not_ratios.most_common()))[0:30])

[('verb', -4.605170185988092), ('phonology', -4.605170185988092), ('syntax', -4.605170185988092), ('translation', -4.605170185988092), ('semantic', -4.605170185988092), ('sentence', -4.605170185988092), ('workshop', -4.605170185988092), ('discourse', -4.605170185988092), ('linguist', -4.605170185988092), ('abstract', -4.605170185988092), ('grammar', -4.605170185988092), ('theory', -4.605170185988092), ('linguistic', -4.605170185988092), ('french', -3.983481969410291), ('linguistics', -3.9670827826111736), ('spanish', -3.9314410912852478), ('august', -3.921875301871158), ('dictionary', -3.916985794770275), ('science', -3.8833378574292876), ('language', -3.8082548198206565), ('japanese', -3.8011223093947044), ('speaker', -3.776477513431922), ('german', -3.7462779928199685), ('structure', -3.6346396851769396), ('english', -3.586982690639934), ('query', -3.526166368708358), ('et', -3.519715981897493), ('papers', -3.5109826833003375), ('ftp', -3.506557897319982), ('1995', -3.464885200919413

Get unique words as features in the dataframe

In [284]:
word_array = set(total_counts.keys())
print(len(word_array ))

27336


In [285]:
word_numpy_array = np.zeros((1, len(word_array)))
print(word_numpy_array.shape)
print(word_numpy_array)

(1, 27336)
[[0. 0. 0. ... 0. 0. 0.]]


In [286]:
#  Maps a word to its column in the word array
word_column_dict = {}
new_word_column_dict={}
for i, word in enumerate(word_array):
    # {key: value} is {word: column}
    word_column_dict[word] = i
c=0
for k,v in word_column_dict.items():
    if v>=1500:
        new_word_column_dict[k]=v
        c=c+1
print(c)        

25836


In [287]:
sorted_IG={}
sorted_new_word_column_dict = sorted(new_word_column_dict.items(), key=lambda kv: kv[1])
c=0
#for k,v in reversed(sorted_new_word_column_dict):
#         print(k,v)
#         c=c+1
#print('length :',c)
c=0
attri_dict={}
for k,v in reversed(sorted_new_word_column_dict):
    if c<=100:
        attri_dict[k]=v
        c=c+1
for k,v in attri_dict.items():
    print(k,v)

technopole 27335
computer-assist 27334
fertig 27333
45-14 27332
unspecify 27331
controversy 27330
6-210 27329
vuw 27328
calls 27327
negotiation 27326
visit 27325
a321 27324
flege 27323
yong 27322
webcrawler 27321
sydow 27320
5108 27319
neuroanatomical 27318
maus 27317
univer 27316
coling-92 27315
673366 27314
cz 27313
nunberg 27312
undergo 27311
nyankore

datum 27310
yoruba 27309
cryptosporidium 27308
deseret 27307
declinable 27306
c-acute 27305
eureka 27304
nun 27303
mostly 27302
kw 27301
20057-1082 27300
11225 27299
eluerd 27298
000yen 27297
sublexical 27296
euralex96

content 27295
metatheory 27294
western 27293
keli 27292
endrike 27291
absolutly 27290
commander 27289
walon 27288
solace 27287
94530 27286
fieldable 27285
service

please 27284
uio 27283
organisational 27282
woyw 27281
forth 27280
winfry 27279
rich 27278
dopke 27277
table 27276
nicholson 27275
intern 27274
systemic 27273
seaport 27272
igo 27271
articulate 27270
ryukyuan 27269
hyderabad 27268
svartvik 27267
paolum 27266

In [288]:
att_values=[]
for k, v in attri_dict.items():
    att_values=[]
    for item in train_df['tokenized_text']:
        if k in item:
            att_values.append(1)
        else:
            att_values.append(0)
    train_df[k]=att_values

In [289]:
print(train_df.head())

                                                Text     Label  Ind  \
0   here\n\nuniversity degree programs increase p...      spam    1   
1   bisfai deadline extension !\n\nbisfai deadlin...  not_spam    2   
2   incredible opportunity\n\ndear friend , recei...      spam    3   
3   " life without debt "\n\npardon intrusion . o...      spam    4   
4   query :\n\ndear linguist , work double negati...  not_spam    5   

   label_tag                                     tokenized_text  technopole  \
0          1  [university, degree, programs, increase, perso...           0   
1          0  [bisfai, deadline, extension, !, bisfai, deadl...           0   
2          1  [incredible, opportunity, dear, friend, receiv...           0   
3          1  [``, life, without, debt, ``, pardon, intrusio...           0   
4          0  [query, :, dear, linguist, work, double, negat...           0   

   computer-assist  fertig  45-14  unspecify  ...  postwar  utsunomiya  node  \
0                0

In [290]:
#train_df.set_index('Ind', inplace=True)

Add the trained features to test data as well. so that we can test on this data

In [291]:
att_values=[]
for k, v in attri_dict.items():
    att_values=[]
    for item in test_df['tokenized_text']:
        if k in item:
            att_values.append(1)
        else:
            att_values.append(0)
    test_df[k]=att_values

# Start on Ml alogrithm-Neural network -FW BW Propagation

In [292]:
#Convert labels into numpy array
l= train_df['label_tag'].tolist()
Y = np.array(l)
Y.shape

(949,)

In [293]:
#Convert input features into numpy array
X =train_df[train_df.columns[5:]]
X=np.array(X)
X.shape
X.shape[0]

949

In [294]:
#import requird Intiallize weights and bias to random values intitlally based on the size of train data we have 
import time
import sys
feature_set = X
labels = Y 
labels = labels.reshape(X.shape[0],1) 
weights = np.random.rand(X.shape[1], 1)  
bias = np.random.rand(1)  
lr = 0.05  

In [295]:
#Define sifmoid function which is activate function for nn
def sigmoid(x):  
    return 1/(1+np.exp(-x))
def sigmoid_der(x):  
    return sigmoid(x)*(1-sigmoid(x))

In [296]:
#FORWARD AND BACKWARD PROPAGATION METHOD STARTS HERE epoche values is given maunally.
for epoch in range(20000):  
    inputs = feature_set

    # forward path
    XW = np.dot(feature_set, weights) + bias

    #forward path 2
    z = sigmoid(XW)

    # backpropagation find the error rate
    error = z - labels

    print(error.sum())
    # backpropagation  based on the error rate
    dcost_dpred = error
    dpred_dz = sigmoid_der(z)

    z_delta = dcost_dpred * dpred_dz

    inputs = feature_set.T
    weights -= lr * np.dot(inputs, z_delta)
#Calculate the bias
    for num in z_delta:
        bias -= lr * num

208.3183518651166
-266.02273412056655
355.7235440497668
-307.83967556800417
367.437935257772
-291.0925010046487
347.600610385555
-286.6390908313409
337.3120051453675
-281.1822286767655
328.75637536340287
-276.80275692665447
322.32680417403753
-273.22743949562494
317.29223100505357
-270.2598408022757
313.20017268769675
-267.73619340668813
309.76088960399306
-265.5401485161251
306.7922775952903
-263.59360313091736
304.1784271923518
-261.8446384549942
301.8431441332469
-260.2578594194648
299.73392184723195
-258.8078117105181
297.81252649171813
-257.47493847875194
296.04960741609364
-256.2433108918323
294.4216740640924
-255.09944980733462
292.9094032494173
-254.03174086287788
291.4966579220903
-253.03013271285772
290.1698704521784
-252.08595109409902
288.91760962001877
-251.19175146472102
287.73024395204834
-250.3411814249057
286.59966212479316
-249.52884639836552
285.51903368912804
-248.7501802966181
284.4826028473451
-248.00132466222232
283.4855114607138
-247.27901909719864
282.523648431

In [297]:
#slope = input x dcost_dpred x dpred_dz 
z_delta = dcost_dpred * dpred_dz  
inputs = feature_set.T  
weights -= lr * np.dot(inputs, z_delta) 

In [298]:
print(len(test_df))

299


In [299]:
#Testing the algorithm on test dataframe we have prediction results displayed for each entry also we have deifned ones with <40% will be non spam and others will be spam
for i in range(len(test_df)):
    list1=test_df.iloc[i]
    list2=list1[5:]
    X =list2
    X=np.array(X,dtype=object)  
    X = X.astype(float)
    result = sigmoid(np.dot(X, weights) + bias) 
    if result >=0.4:
         print(result ,":" "Spam") 
    else:
        print(result ,":" "Not spam") 
    

[0.66045564] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.46899958] :Spam
[0.66045564] :Spam
[0.00157057] :Not spam
[0.3119233] :Not spam
[0.97808166] :Spam
[0.51596341] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.97808166] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.90154071] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.9996156] :Spam
[0.08843722] :Not spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.97808166] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.93767305] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.00062548] :Not spam
[0.97808166] :Spam
[0.66045564] :Spam
[0.97808166] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.9996156] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.66045564] :Spam
[0.93767305] :Spam
[0.87230796] :Spam
[0.66045564] :Spam
[1.93089525e-08] :Not spam
[0.6604556